In [1]:
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Damage
import numpy as np
from time import sleep, time

# Collision Avoidance

## Setting up Simulation Parameters 

In [2]:
beamng = BeamNGpy('localhost', 64256, user='D:/ros', home='D:/BeamNG/game')
bng = beamng.open(launch=False)
bng.set_steps_per_second(50)
bng.set_deterministic()

## Scenario Definition

In [3]:
scenario = Scenario('west_coast_usa', 'SBST: Intersection')

ego = Vehicle('ego', model='etk800', color='black')
damage = Damage()
ego.attach_sensor('damage', damage)
pos = (-717.12, 100.90, 118.65)
rot = (0, 0, 0.919, -0.395)
scenario.add_vehicle(ego, pos=pos, rot=None, rot_quat=rot)

obstacle = Vehicle('obstacle', model='etk800', color='white')
damage = Damage()
obstacle.attach_sensor('damage', damage)
pos = (-698.61, 313.43, 131.71)
rot = (-0.082, -0.031, -0.410, 0.908)
scenario.add_vehicle(obstacle, pos=pos, rot=None, rot_quat=rot)
scenario.make(bng)

## Scenario Execution

In [4]:
bng.load_scenario(scenario)
bng.start_scenario()

meter_per_second = 16

bng.pause()
bng.switch_vehicle(ego)
ego.ai_set_mode('manual')
ego.ai_set_waypoint('wpTown_6')
ego.ai_set_waypoint('wpTown_18')
ego.ai_set_speed(meter_per_second)
bng.resume()

sleep(3)

bng.pause()
obstacle.ai_set_mode('manual')
obstacle.ai_set_waypoint('wpTown_2')
obstacle.ai_set_waypoint('wpTown_3')
obstacle.ai_set_speed(meter_per_second)
bng.resume()

### Ego Vehicle Emergency Break

In [5]:
velocity = meter_per_second
ego.ai_set_mode('disabled')
ego.control(brake=1)
while velocity > 0.1:
    ego.poll_sensors()
    velocity = np.linalg.norm(ego.sensors['state'].data['vel'])
ego.control(brake=0)

## Damage Based Test Oracle

In [6]:
obstacle.poll_sensors()
ego.poll_sensors()

d = ego.sensors['damage'].data['damage']
print(f'ego vehicle damage: {d}')
d = obstacle.sensors['damage'].data['damage']
print(f'obstacle vehicle damage: {d}')


ego vehicle damage: 0
obstacle vehicle damage: 0
